In [ ]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

import time
import json
import argparse

import tensorflow as tf
import numpy as np
import cv2
from glob import glob

from eval import evaluation_model_by_mAP

from tensorflow.keras.layers import Lambda
from yolov3.models import YoloV3, yolo_boxes, yolo_nms
from yolov3.anchors import default_anchor
from yolov3.dataset import transform_images
from yolov3.utils import draw_outputs

anchors = default_anchor/416

In [ ]:
model_loaded = tf.saved_model.load('./ETC/export_batch')

In [ ]:
def predict_batch(model, images, nms_config=(300,0.5,0.5)):
    # model : exported model (nms 이전 단계에서 export 된 모델)
    # images : batch images, shape = (batch, height, width, channel)
    
    n_batch = images.shape[0]
    outputs = []
    
    b_0, b_1, b_2 = model(images)
    
    for i in range(n_batch):
        boxes, scores, classes, nums = b_0
        b0 = (boxes[i:i+1,...],scores[i:i+1,...],classes[i:i+1,...], nums[i:i+1,...])
        
        boxes, scores, classes, nums = b_1
        b1 = (boxes[i:i+1,...],scores[i:i+1,...],classes[i:i+1,...], nums[i:i+1,...])
        
        boxes, scores, classes, nums = b_2
        b2 = (boxes[i:i+1,...],scores[i:i+1,...],classes[i:i+1,...], nums[i:i+1,...])
        
        output = (lambda x: yolo_nms(x, 5, nms_config))((b0[:3], b1[:3], b2[:3]))
        
        outputs.append(output)
        
    return outputs

# 

# Batch Predict Test

In [ ]:
img_1 = cv2.imread('./ETC/sample_1.jpg')
img_2 = cv2.imread('./ETC/sample_2.jpg')

In [ ]:
img_batch = np.zeros((2,800,800,3))
img_batch[0,:,:,:] = img_1
img_batch[1,:,:,:] = img_2

img_batch = transform_images(img_batch,800)

In [ ]:
# batch prediction
outputs = predict_batch(model_loaded, img_batch, nms_config=(300,0.5,0.5))

for idx, output in enumerate(outputs):
    boxes, scores, classes, nums = output

    draw_img = draw_outputs(np.array(img_batch[idx,:,:,:]*255,dtype=np.uint8), (boxes, scores, classes, nums), str(classes))
    cv2.imwrite('./ETC/{}_result.png'.format(str(idx)), draw_img)

# 

# Batch 시간 비교

In [ ]:
import time
from collections import defaultdict
from tqdm import tqdm

In [ ]:
n_test_images = 1000
n_test_case = 5

n_batch = [1,8,16,32,64]
predict_time = defaultdict(list)

In [ ]:
model_loaded = tf.saved_model.load('./ETC/export_batch')

In [ ]:
# warm-up iterateration
dummy_img = np.zeros((1,800,800,3))
dummy_img = transform_images(dummy_img,800)
for i in range(100): model_loaded(dummy_img)

In [ ]:
for test_num in range(n_test_case):
    print("test case : {}".format(str(test_num)))
    for batch in n_batch:
        s_time = time.time()
        image_batch = np.zeros((batch,800,800,3))
        image_batch = transform_images(image_batch,800)
        
        for _ in range(n_test_images//batch):
            model_loaded(image_batch)
        
        if n_test_images%batch !=0 : 
            image_batch = np.zeros((n_test_images%batch,800,800,3))
            image_batch = transform_images(image_batch,800)
            model_loaded(image_batch)
            
        end_time = time.time()-s_time
        predict_time[batch].append(end_time)
        print(batch, end_time)
    print("\n\n")

In [ ]:
import statistics
for batch in n_batch:
    print("batch : {}, time : {}".format(str(batch),str(statistics.median(predict_time[batch]))))

# 

# 모델 구조 확인

In [ ]:
!saved_model_cli show --dir /home/jovyan/jingwan/YOLOV3_TF/ETC/export_test/ --tag_set serve --signature_def serving_default

In [ ]:
!saved_model_cli show --dir /home/jovyan/jingwan/YOLOV3_TF/ETC/export_original/ --tag_set serve --signature_def serving_default